In [1]:
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-tib-core.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='models/mpnet_gnd_triplet' #"gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

# import pickle
# with open('./tmp/core_labels_mpnet.pkl','wb') as f:
#     pickle.dump(y_t,f)

# 打印当前时间
import datetime
print(datetime.datetime.now())

2025-01-19 15:08:02.414617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737270482.426547   95577 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737270482.429982   95577 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 15:08:02.443832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensor([[ 0.0257,  0.1114, -0.0401,  ..., -0.1732, -0.1499, -0.4778],
        [ 0.0266,  0.1118, -0.0426,  ..., -0.1720, -0.1483, -0.4785],
        [ 0.0261,  0.1116, -0.0411,  ..., -0.1726, -0.1490, -0.4780]],
       device='cuda:0', dtype=torch.float16)
79427 79427
2025-01-19 15:08:35.115067


In [2]:
import os
import json
import random

root='/media/jh/新加卷/2024_12_04/llms4subjects-main/shared-task-datasets/TIBKAT/'
all_core='tib-core-subjects'
def process_data(root,dev,en_de,datas):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'subject' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])!=str:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])!=str:
            item['abstract']=' '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        item['dcterms:subject']=tmp['dcterms:subject']
        item['subject']=tmp['subject']
        datas.append(item)
    for dirs in os.listdir(root+f'{all_core}/data/{dev}'):
        for file in os.listdir(root+f'{all_core}/data/{dev}/{dirs}/{en_de}'):
            get_train_data(root+f'{all_core}/data/{dev}/{dirs}/{en_de}/{file}')
     
train_datas = []
en_de='en'
dev='train'
process_data(root,dev,en_de,train_datas)
en_de='de'
dev='train'
process_data(root,dev,en_de,train_datas)
random.shuffle(train_datas)

# dev_datas=[]
# en_de='en'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# en_de='de'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# random.shuffle(dev_datas)

import json
root='/media/jh/新加卷'
file_path=f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())

print(data[0])
gnd_id2name={}
for i in data:
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+' '+j
print(i['Code'],gnd_id2name[i['Code']])


import json

print('len(train_datas)=',len(train_datas))
# print('len(dev_datas)=',len(dev_datas))
datas=train_datas
print('len(datas)=',len(datas))
# datas数据样本随机打乱
import random
print(datas[0])
random.shuffle(datas)
print(datas[0])


from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch


# with open('./tmp/core_labels_gte.pkl','rb') as f:
#     y_t=pickle.load(f)

model_name_or_path='models/mpnet_gnd_triplet' #"gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)

def process_data(datas,result):
    batch_size=768
    texts=[datas[i:i+batch_size] for i in range(0, len(datas), batch_size)]
    if len(texts)*batch_size<len(datas):
        texts.append([datas[len(texts)*batch_size:]])
    for block in texts:
        text=[t['title']+'.'+t['abstract'] for t in block]
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
        with torch.no_grad():
            similarities=torch.tensor(similarities).to('cuda')
            # print(similarities.shape) 上次保存的前500个
            top_k = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最高的20个
            #similarities中的元素前500个取反
            similarities=-similarities[:500]
            top_k_last = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最低的20个
        for i,t in enumerate(block):
            pos=[]
            for gnd in t['dcterms:subject']:
                if type(gnd)==dict:
                    gnd_id=gnd['@id']
                else:
                    continue
                if gnd_id in gnd_id2name:
                    pos.append(gnd_id2name[gnd_id])
            for j in top_k[i]:
                pos.append(gnd_id2name[y_t[j]['Code']])
            item1={}
            item1['pos']='.'.join(pos)
            item1['neg']='. '.join([gnd_id2name[y_t[j]['Code']] for j in top_k_last[i]])
            item1['text']=t['title']+'.'+t['abstract']
            result.append(item1)

# 打印运行时间
import time
start_time = time.time()
x_train=[]
process_data(datas,x_train) 
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

{'Code': 'gnd:4003694-7', 'Classification Number': '00', 'Classification Name': 'Unspezifische Allgemeinwörter', 'Name': 'Ausbreitung', 'Alternate Name': [], 'Related Subjects': [], 'Source': 'Du.'}
gnd:1163295434 Airfix Marke
len(train_datas)= 41902
len(datas)= 41902
{'title': 'Jumpstart Tableau : A Step-By-Step Guide to Better Data Visualization', 'abstract': 'Chapter 1: Logon to Tableau -- Chapter 2: Connecting to Two Data Sources -- Chapter 3: Exploring the Tableau Interface -- Chapter 4: Developing a Simple Visualization -- Chapter 5: Saving Tableau Workbook and Tableau Packaged Workbook -- Chapter 6: Using Basic Analysis Functions -- Chapter 7: Adding, Removing, and Renaming a Dimension -- Chapter 8: Copying or Deleting a Worksheet -- Chapter 9: Changing the Display from One Chart Type to Another -- Chapter 10: Using the Show Me Tool for Selecting a Visualization -- Chapter 11: Crosstab Display and Swapping -- Chapter 12: Sorting -- Chapter 13: More about Sorting -- Chapter 14: V

In [4]:
import pickle
with open('./tmp/train_core_mpnet.pkl','wb') as f:
    pickle.dump(x_train,f)

In [ ]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷1'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='models/gte_gnd'#'/media/jh/新加卷/2025_1_16/sentence-transformers/all-mpnet-base-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷1'
root='/media/jh/新加卷1/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷1/2024_12_04/2025/gte_gnd_test'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

In [7]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='/media/jh/新加卷/2025_1_16/all-MiniLM-L6-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷'
root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷/2024_12_04/run-all-MiniLM-L6-v2'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[-0.0220, -0.0606, -0.0200,  ...,  0.0100, -0.0632,  0.0414],
        [-0.0707, -0.0256, -0.0237,  ...,  0.0167, -0.0672, -0.0280],
        [-0.0512, -0.0390,  0.0042,  ..., -0.0457, -0.1104,  0.0361]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 9996
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 10752
batch 11520
batch 12288
batch 13056
batch 13824
batch 14592
batch 15360
batch 16128
batch 16896
batch 17664
batch 17990
运行时间: 1043.0688846111298 秒


In [8]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='/media/jh/新加卷/2025_1_16/sentence-transformers/all-distilroberta-v1/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷'
root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷/2024_12_04/run-all-distilroberta-v1'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[ 0.0049, -0.0529,  0.0514,  ...,  0.0145,  0.0130,  0.0009],
        [-0.0096, -0.0480,  0.0023,  ...,  0.0433, -0.0103,  0.0004],
        [-0.0368, -0.0287,  0.0128,  ...,  0.0311, -0.0169,  0.0010]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 9996
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 10752
batch 11520
batch 12288
batch 13056
batch 13824
batch 14592
batch 15360
batch 16128
batch 16896
batch 17664
batch 17990
运行时间: 1415.5852499008179 秒


In [9]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='/media/jh/新加卷/2025_1_16/sentence-transformers/all-mpnet-base-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷'
root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷/2024_12_04/run-all-mpnet-base-v2'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[-0.0133, -0.0674, -0.0301,  ...,  0.0318, -0.0150, -0.0321],
        [-0.0133, -0.0674, -0.0301,  ...,  0.0318, -0.0150, -0.0321],
        [-0.0133, -0.0674, -0.0301,  ...,  0.0319, -0.0150, -0.0321]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 9996
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 10752
batch 11520
batch 12288
batch 13056
batch 13824
batch 14592
batch 15360
batch 16128
batch 16896
batch 17664
batch 17990
运行时间: 1495.6767885684967 秒


In [10]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='/media/jh/新加卷/2025_1_16/sentence-transformers/paraphrase-MiniLM-L6-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷'
root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷/2024_12_04/run-paraphrase-MiniLM-L6-v2'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[-0.2419,  0.8901, -0.3267,  ...,  0.0905, -0.6177, -0.5166],
        [-0.4568,  0.6929, -0.4431,  ...,  0.1571, -0.6353, -0.3936],
        [-0.4717,  0.7520, -0.5210,  ...,  0.2507, -0.7920, -0.3259]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 9996
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 10752
batch 11520
batch 12288
batch 13056
batch 13824
batch 14592
batch 15360
batch 16128
batch 16896
batch 17664
batch 17990
运行时间: 1077.915846824646 秒


In [11]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='/media/jh/新加卷/2025_1_16/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷'
root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/test/'
des_root='/media/jh/新加卷/2024_12_04/run-paraphrase-multilingual-MiniLM-L12-v2'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[-0.0318,  0.1566,  0.0232,  ...,  0.2311,  0.1600,  0.0582],
        [-0.1687,  0.1361,  0.0512,  ...,  0.0435,  0.0306,  0.0728],
        [-0.0496,  0.3281, -0.0147,  ..., -0.0675, -0.0553,  0.1239]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 9996
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7680
batch 8448
batch 9216
batch 9984
batch 10752
batch 11520
batch 12288
batch 13056
batch 13824
batch 14592
batch 15360
batch 16128
batch 16896
batch 17664
batch 17990
运行时间: 1034.0145630836487 秒


In [14]:
m2_similarities=[]
with open('./tmp/m2_en.pkl', 'rb') as f:
    m2_similarities = pickle.load(f)   
print(torch.tensor(m2_similarities).shape)

torch.Size([6045, 79427])


In [6]:
# m2_similarities=[]
# with open('./tmp/m2_de.pkl', 'rb') as f:
#     m2_similarities = pickle.load(f)
len(m2_similarities[0])

768

In [1]:
# 生成all标签嵌入向量 int8量化
import os
import json

root='/media/jh/新加卷'
# model_id='/media/jh/新加卷/2024_12_04/gnd_model_result'
model_id='/media/jh/新加卷/2024_12_04/gnd_models'
with open(f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
from sentence_transformers.quantization import quantize_embeddings
import torch


model = SentenceTransformer(model_id)
model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True,precision="binary")
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

import pickle
with open('./tmp/all_labels_int8.pkl','wb') as f:
    pickle.dump(y_t,f)

2025-01-17 09:20:24.653957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737076824.666171  100228 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737076824.670215  100228 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 09:20:24.683252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensor([[  28,  -16,  -73,  -84,  -50, -111,   59,  -43,   50,  -90,   88,   60,
          -75,  126,  124,   25,   28,  -91,  -83,  -59,   98,  -49,  108, -120,
          109, -103,  -57, -120, -102,  102,  -96, -118,   10,   65,  -99,  -50,
           83,  -54,  118,  -10,   44,  111,  -47,  -39,   65,  -17,   72,   86,
          -48,  -59,   -5,   84,  -38,   55,  -54,    0,   70,   56,  120,  126,
          -85,   71,  -91,  101,  107,  -24,   -1,  -76,   14,  -63,   40,  107,
          125,  -52, -117,    7, -115, -106,   83,   93,   45,  -52,   20,  -94,
           82, -124,   -6,  -16,  -32,   96,   28,  102,  -44,   65,   11,   -9],
        [  28,   -4,  -11,  -90,  -56, -111,   59,  -11,   62,  -74,  -56,   28,
          -99,  -82,  127,   37,  124,  -83,  -83,  -59,  -32,   79,    8,    8,
          101,  -71,   71, -120,   30,   98,  -90, -102,   11,   64, -107,  -86,
          -46,  -62,  -12,  -74,   38,  103,  -16,   65,   73,  -25,   72,   68,
          116,  -43,   -7, 

In [2]:
from sentence_transformers import SentenceTransformer
import torch
import os
import json
import pickle

# model_id='/media/jh/新加卷/2024_12_04/gnd_model_result'
model_id='/media/jh/新加卷/2024_12_04/gnd_models'
en_de='de'
root='/media/jh/新加卷/2024_12_04/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/dev'
des_root='test/test_core1'

with open('./tmp/all_labels_int8.pkl','rb') as f:
    y_t=pickle.load(f)

model = SentenceTransformer(model_id) #sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
model=model.to('cuda')

def process_data(en_de,root,des_root):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
                continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True, precision="binary")
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')

            similaritie = model.similarity(y_tt.to(torch.float32),embeddings.to(torch.float32)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
        
        similarities=torch.tensor(similarities).to('cuda')
        # print(similarities.shape) 上次保存的前500个
        top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
        for t in top_k:
            code=[]
            for i in t:
                code.append(y_t[i]['Code'])
            item1={}
            item1['dcterms:subject']=code
            result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            if os.path.exists(file_names[index]):
                index+=1
                continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
    # print('数据处理完成，开始写入文件')
    # for i in range(len(result)):
    #     # 如果不存在目录，则创建目录
    #     if os.path.exists(os.path.dirname(file_names[i])):
    #         continue
    #     if not os.path.exists(des_path[i]):
    #         os.makedirs(des_path[i])
    #     with open(file_names[i],'w') as f:
    #         json.dump(result[i],f,ensure_ascii=False,indent=4)

process_data('en',root,des_root) #15904
process_data('de',root,des_root) #24165 24min

batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6045
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7621


In [17]:
# tmp=[torch.zeros(3,5),torch.zeros(3,5),torch.zeros(3,5),torch.zeros(2,5)]
# tmp1=[]
# for i in range(4):
#     tmp1.extend(tmp[i].tolist())
# tmp1=torch.tensor(tmp1)
# print(tmp1.shape)
import torch
import torch.nn.functional as F
def cosine_similarity_custom(tensor1, tensor2):
    # 计算两个张量的点积
    dot_product = torch.sum(tensor1 * tensor2 ,dtype=tensor1.dtype)
    
    # 计算两个张量的范数（L2范数）
    norm1 = torch.sqrt(torch.sum(tensor1 ** 2, dtype=tensor1.dtype))
    norm2 = torch.sqrt(torch.sum(tensor2 ** 2, dtype=tensor2.dtype))
    
    # 计算余弦相似度
    cosine_sim = dot_product / (norm1 * norm2)
    
    return cosine_sim
# 创建两个 int8 类型的张量
tensor1 = torch.tensor([[1, 2, 3]], dtype=torch.int8)
tensor2 = torch.tensor([4, 5, 6], dtype=torch.int8)

cos_sim = cosine_similarity_custom(tensor1, tensor2)

print(type(cos_sim.item()))
print(cos_sim.dtype)

<class 'float'>
torch.float32


In [1]:
# 生成core标签嵌入向量
import os
import json

root='/media/jh/新加卷'
with open(f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-tib-core.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

model_id='/media/jh/新加卷/2024_12_04/gnd_model_result'

model = SentenceTransformer(model_id) #sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

import pickle
with open(f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/core_labels.pkl','wb') as f:
    pickle.dump(y_t,f)

2025-01-13 21:32:13.777301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736775133.950706   24950 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736775133.989734   24950 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 21:32:14.370376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensor([[ 0.0146, -0.1317, -0.0023,  ...,  0.1478,  0.0060, -0.1274],
        [ 0.0770, -0.0802, -0.0041,  ..., -0.0109, -0.0813, -0.1333],
        [-0.0050, -0.0863, -0.0048,  ...,  0.0784, -0.0179, -0.1176]],
       device='cuda:0')
79427 79427


In [ ]:
import torch.nn.functional as F
import os
import json
import pickle
root1='/media/jh/新加卷'
with open(f'{root1}/2024_12_04/llms4subjects-main/shared-task-datasets/core_labels.pkl','rb') as f:
    y_t=pickle.load(f)
from sentence_transformers import SentenceTransformer
import torch

model_id='/media/jh/新加卷/2024_12_04/gnd_model_result'

model = SentenceTransformer(model_id) 
model=model.to('cuda')


def process_data(en_de,root,des_root):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
                continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=64
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        result=[] #重置result
        for embedding in embeddings:
            with torch.no_grad():
                cosine_similarity = F.cosine_similarity(embedding, y_tt, dim=1)
                top_k = torch.topk(cosine_similarity, k=50, dim=0) # 取出相似度最高的50个
                top_k=top_k.indices.cpu().tolist()
            code=[]
            for i in top_k:
                code.append(y_t[i]['Code'])
            item1={}
            item1['dcterms:subject']=code
            result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            if os.path.exists(file_names[index]):
                index+=1
                continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
    # print('数据处理完成，开始写入文件')
    # for i in range(len(result)):
    #     # 如果不存在目录，则创建目录
    #     if os.path.exists(os.path.dirname(file_names[i])):
    #         continue
    #     if not os.path.exists(des_path[i]):
    #         os.makedirs(des_path[i])
    #     with open(file_names[i],'w') as f:
    #         json.dump(result[i],f,ensure_ascii=False,indent=4)
en_de='de'
root='./test-tib-core'
des_root=f'{root1}/2024_12_04/result_core'
process_data('en',root,des_root)
process_data('de',root,des_root)